In [248]:
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import numpy as np
import html5lib
import requests
from openpyxl import Workbook
from datetime import datetime
import webbrowser
import os

In [249]:
path = '../Test_GBDC_Investment.xlsx'
xls = pd.ExcelFile(path)
all_sheets = pd.read_excel(path, sheet_name=None)
dataframes = {}
# Loop through each sheet and create a DataFrame in the dictionary
for sheet_name, sheet_df in all_sheets.items():
    dataframes[sheet_name.replace(' ', '_')] = sheet_df
    print(f"DataFrame name: {sheet_name} : {sheet_df.shape}")

DataFrame name: March 31 2013 : (424, 31)
DataFrame name: June 30 2013 : (432, 31)
DataFrame name: September 30 2013 : (486, 41)
DataFrame name: December 31 2013 : (443, 31)
DataFrame name: March 31 2014 : (467, 31)
DataFrame name: June 30 2014 : (482, 27)
DataFrame name: September 30 2014 : (527, 33)
DataFrame name: December 31 2014 : (506, 31)
DataFrame name: March 31 2015 : (524, 31)
DataFrame name: June 30 2015 : (571, 31)
DataFrame name: September 30 2015 : (584, 33)
DataFrame name: December 31 2015 : (505, 27)
DataFrame name: March 31 2016 : (589, 27)
DataFrame name: June 30 2016 : (564, 27)
DataFrame name: September 30 2016 : (700, 23)
DataFrame name: December 31 2016 : (646, 23)
DataFrame name: March 31 2017 : (677, 23)
DataFrame name: June 30 2017 : (699, 23)
DataFrame name: December 31 2017 : (734, 41)
DataFrame name: March 31 2018 : (766, 25)
DataFrame name: June 30 2018 : (817, 25)
DataFrame name: September 30 2018 : (883, 24)
DataFrame name: December 31 2018 : (916, 41)
Da

In [358]:
def process_table_fun(soi_table_df, process_tables_shapes):
    print(1, 'shape:', soi_table_df.shape)
    soi_table_df = soi_table_df.replace(r'^\s*\$\s*$', np.nan, regex=True)
    soi_table_df = soi_table_df.replace(r'\n', '', regex=True)
    print(2, 'shape:', soi_table_df.shape)
    # fewer non-null values, effectively dropping rows with more than 100 non-null values.
    # for index, row in soi_table_df.iterrows():
    #     if row.count() > 100:
    #         soi_table_df = soi_table_df.drop(index)

    pattern = r'Total\s+Investments'
    # Use the apply function to check if the pattern is in any column for each row
    matching_rows = soi_table_df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
    # Find the index of the first row that matches the pattern
    # Slice the DataFrame to keep only the rows up to and including the first matching row
    if soi_table_df[matching_rows].index[0] < 20:
        soi_table_df = soi_table_df.loc[:soi_table_df[matching_rows].index[1]].reset_index(
            drop=True)
    else:
        soi_table_df = soi_table_df.loc[:soi_table_df[matching_rows].index[0]].reset_index(
            drop=True)

    print(0, 'shape:', soi_table_df.shape)
    soi_table_df = soi_table_df.dropna(how='all', axis=1).reset_index(
        drop=True)

    print(3, 'shape:', soi_table_df.shape)
    soi_table_df = soi_table_df.dropna(how='all', axis=0).reset_index(
        drop=True)

    print(4, 'shape:', soi_table_df.shape)
    soi_table_df.dropna().reset_index(
        drop=True)
    print(5, 'shape:', soi_table_df.shape)

    # pattern = r'(?i)investment\s*type?|investment'
    # pattern = 'Investments'
    # pattern = r'Net asset value per common share'
    # matching_rows = soi_table_df.apply(
    #     lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)

    # if matching_rows is not None:
    #     soi_table_df = soi_table_df.loc[soi_table_df[matching_rows].index[0]:].reset_index(
    #         drop=True)

    pattern = r'Net asset value per common share|How We Addressed the Matter in Our Audit'
    matching_rows = soi_table_df.apply(
        lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)

    # Check if the pattern exists in the DataFrame
    if matching_rows.any():
        # Extract rows from the first occurrence onwards
        soi_table_df = soi_table_df.iloc[matching_rows.idxmax(
        )+1:].reset_index(drop=True)
        # Rename columns with the values from the first row
    soi_table_df = soi_table_df.rename(
        columns=soi_table_df.iloc[0]).drop(soi_table_df.index[0])

    # Reset the index after making modifications
    soi_table_df.reset_index(drop=True, inplace=True)

    # second_row_values = soi_table_df.iloc[1]
    # for col_idx, value in enumerate(second_row_values):
    #     if not pd.isna(value) and not pd.isna(soi_table_df.columns[col_idx]):
    #         col_name = soi_table_df.columns[col_idx]
    #         new_col_name = f"{col_name} {str(value)}"
    #         soi_table_df.rename(columns={col_name: new_col_name}, inplace=True)
    #         soi_table_df.iloc[2, col_idx] = np.nan

    combined_row = soi_table_df.iloc[0:3].apply(
        lambda x: ', '.join(x.dropna()), axis=0)

    # Drop the original rows and insert the combined row at the beginning
    soi_table_df = soi_table_df.drop(index=soi_table_df.index[0:3])
    soi_table_df = pd.concat(
        [combined_row, soi_table_df]).reset_index(drop=True)

    print(6, 'shape:', soi_table_df.shape)
    soi_table_df = soi_table_df.replace('—', 0)
    soi_table_df = soi_table_df.replace('-', np.nan)
    print(7, 'shape:', soi_table_df.shape)

    soi_table_df = soi_table_df.applymap(
        lambda x: x.strip() if isinstance(x, str) else x)

    process_tables_shapes.append(soi_table_df.shape)

    return soi_table_df

In [359]:
path = '/Users/fuadhassan/Desktop/BDC_RA/GBDC/process_tables_GBDC_Investment.xlsx'
writer = pd.ExcelWriter(path, engine='openpyxl')
process_tables = {}
process_tables_shape = []
for dataframe in dataframes:
    print(dataframe)
    process_tables[dataframe] = process_table_fun(
        dataframes[dataframe], process_tables_shape)
    print('\n')
    process_tables[dataframe].to_excel(
        writer, sheet_name=dataframe.replace(',', ''), index=False)
    writer.book .save(path)
writer.close()

March_31_2013
1 shape: (424, 31)
2 shape: (424, 31)
0 shape: (424, 31)
3 shape: (424, 17)
4 shape: (424, 17)
5 shape: (424, 17)


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [355]:
process_tables['March_31_2013'].to_csv('test.csv')

In [306]:
soi_table_df = process_tables['March_31_2013']

In [307]:
soi_table_df

,0,2,4,5,7,9,10,13,16,17,19,21,22,25,26,29,30
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Percentage,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Investment,Spread Above,NaN,Interest,NaN,Maturity,Principal,NaN,NaN,of,NaN,Fair,NaN,NaN,NaN,NaN
2,NaN,Type,Index (1),NaN,Rate (2),NaN,Date,Amount,Cost,NaN,Net Assets,NaN,Value,NaN,NaN,NaN,NaN
3,Investments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Canada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,Vision Source L.P.,Common stock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5389,NaN,1.2,NaN,5667,NaN
421,Total equity investments United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23540,NaN,5.2,%,26023,NaN
422,Total United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,780136,NaN,157.6,%,787520,NaN


In [254]:

pattern = r'(?i)investment\s*type?|investment'
matching_rows = soi_table_df.apply(
    lambda row: row.str.contains(pattern, flags=re.IGNORECASE, regex=True).any(), axis=1)
soi_table_df = soi_table_df.loc[soi_table_df[matching_rows].index[0]:].reset_index(
    drop=True)

In [255]:
df = soi_table_df

# Define the regular expression pattern to match the variations
pattern = r'(?i)investment\s*type?|investment'

# Check if any rows match the pattern
matching_rows = soi_table_df[~soi_table_df.astype(str).apply(lambda x: x.str.contains(
    pattern, flags=re.IGNORECASE, regex=True))['Investment']]
matching_rows

KeyError: 'Investment'

In [ ]:
matching_rows

In [ ]:
pattern = r'(?i)investment\s*type?|investment'

# Create a boolean mask for rows where the pattern doesn't match
no_match_mask = ~soi_table_df.astype(str).apply(
    lambda x: x.str.contains(pattern, flags=re.IGNORECASE, regex=True)).any(axis=1)

# Filter rows based on the mask
no_match_mask

In [ ]:
import pandas as pd

# Sample DataFrame
data = {'A': [1, 2, 3],
        'B': [4, 5, 6],
        'C': [7, 8, 9]}

df = pd.DataFrame(data)

# Get the number of columns in the DataFrame
num_columns = len(df.columns)

# Initialize an empty list to store the values of the first row
first_row_values = []

# Loop through column indices and retrieve the values in the first row
for col_idx in range(num_columns):
    value = df.iloc[0, col_idx]
    first_row_values.append(value)

# Print the values of the first row for each column
for col_idx, value in enumerate(first_row_values):
    print(f"Value in column {df.columns[col_idx]} (index {col_idx}): {value}")